In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("nitishabharathi/cert-insider-threat")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\pc\.cache\kagglehub\datasets\nitishabharathi\cert-insider-threat\versions\1


In [3]:
files = os.listdir(path)
print("Dataset Files:", files)
df = pd.read_csv(os.path.join(path, "email.csv"))
# Map Threat column to numerical (Yes -> 1, No -> 0)
df["Threat"] = np.random.choice(["Yes", "No"], size=len(df), p=[0.5, 0.5])
df["Threat"] = df["Threat"].map({"Yes": 1, "No": 0})
df = df.sample(n=500, random_state=42)
# Display first few rows
print(df.head(5))

Dataset Files: ['email.csv', 'psychometric.csv']
                               id                 date     user       pc  \
1375085  {X9O7-L3NQ83SH-7546ROCM}  09/08/2010 09:48:21  HWH0646  PC-8087   
1760687  {N4J5-H0DM81GR-0636AXZX}  11/18/2010 11:40:48  EMW0772  PC-6035   
489761   {T3T3-X0QU63SN-9148FQEX}  03/29/2010 14:40:35  HAD0246  PC-0926   
670295   {S1I7-L4JZ84KI-7061AWTL}  04/30/2010 10:57:44  ASB0796  PC-1130   
2611719  {C8B5-Q0BW49BI-1420XVWZ}  05/12/2011 11:45:23  HCW0419  PC-3772   

                                                        to   cc  \
1375085  Ursa.Britanney.Frederick@dtaa.com;Abel.Adam.Mo...  NaN   
1760687  Butler-Serina@yahoo.com;Vance-Quintessa@netzer...  NaN   
489761                                  TEB582@verizon.net  NaN   
670295   Tobias.Lucas.Morin@dtaa.com;Sonya.Samantha.Hah...  NaN   
2611719                         Herman.Ian.Abbott@dtaa.com  NaN   

                                  bcc                                from  \
1375085       

In [4]:
# Encode categorical variables
categorical_cols = ['id', 'date', 'user', 'pc', 'to', 'cc', 'bcc', 'from', 'content']  # Adjust as needed
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Define features and target
X = df.drop(columns=['Threat']).values  # Features
y = df['Threat'].values  # Target variable


In [5]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)


In [6]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [8]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [9]:
# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim).to(device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_dim).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return self.softmax(out)

In [10]:
# Hyperparameters
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = len(np.unique(y))  # Number of classes
num_layers = 2
num_epochs = 10
learning_rate = 0.001

In [11]:
# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_dim, hidden_dim, output_dim, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
# Training loop
model.train()
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch.unsqueeze(1))
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.6936
Epoch [2/10], Loss: 0.6939
Epoch [3/10], Loss: 0.6920
Epoch [4/10], Loss: 0.6908
Epoch [5/10], Loss: 0.6905
Epoch [6/10], Loss: 0.6879
Epoch [7/10], Loss: 0.6854
Epoch [8/10], Loss: 0.6957
Epoch [9/10], Loss: 0.6884
Epoch [10/10], Loss: 0.7069


In [13]:
# Evaluation
model.eval()
y_pred = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())


In [14]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.5556
Precision: 0.5556
Recall: 0.5556
F1 Score: 0.5556
